# Uncertainty Quantification of Copper Shock Compression Data

This notebook demonstrates uncertainty quantification of a linear model for a copper shock wave-particle velocity data set from Marsh (1980). The demonstrated methods include:

- Linear regression
- Bayesian analysis
    - Computing the posterior distribution in closed-form
    - Sampling the posterior distribution using Markov Chain Monte Carlo (MCMC)
- Bootstrapping

Discussion and extension questions are provided at the end of the notebook to gain a deeper understanding of this material.

In [ ]:
import os

import numpy as np
import pandas as pd

from eosuq import bootstrap, calibration, eda, hugoniot, least_squares, mcmc

In [ ]:
# Load data
data_path = os.path.join("data", "CopperMarsh.csv")
df = pd.read_csv(data_path)

## Exploratory Data Analysis (EDA)

In [ ]:
# Number of points in data set
df.shape[0]

In [ ]:
# Print first few values in dataset
df.head()

In [ ]:
# Two rows are the same
df.iloc[55:57]

In [ ]:
# Plot shock wave versus particle velocity data
eda.plot_raw_data(df)

In [ ]:
# Create a bar plot of the experiment types
eda.plot_experiment_type_distribution(df)

There is an apparent outlier when the particle velocity is approximately 4.1 km/s.  Printing this row indicates that this measurement is the last in the dataset (Python uses zero-based indexing) and that it was produced by a shock and free surface velocity experiment.

In [ ]:
# Print row containing point with largest particle velocity
df[df["Up_km_s"] == df["Up_km_s"].max()]

Let's remove this point from our data set since it appears to be an outlier.

In [ ]:
# Remove outlier with large particle velocity
df = df[df["Up_km_s"] != df["Up_km_s"].max()]

## Linear Regression

Linear regression is the simplest model for the relationship between shock wave and particle velocity.  This model specifies that
$$
U_{s,i}=C_0+SU_{p,i}+\epsilon_i
$$
for $i=1,...,n$, where $U_{s,i}$ and $U_{p,i}$ are the $i$th shock wave and particle velocity, respectively, and the $\epsilon_i$ are independent and identically distributed normal random variables with mean zero and variance $\sigma^2$.  The parameters of this model, $C_0$ and $S$, are constants that require estimation, along with the variance term.

The least squares estimate of $\beta=(C_0,S)'$ is
$$
\hat{\beta}=(X'X)^{-1}X'Y,
$$
where $Y=(U_{s,1},...,U_{s,n})'$ and $X$ is a matrix whose first column is 1's and whose second column is $(U_{p,1},...,U_{p,n})'$.

In [ ]:
# Plot least squares fit with confidence and prediction intervals
least_squares.plot_least_squares(df)

In [ ]:
# Extract coefficients for least squares model
beta_hat, _ = least_squares.compute_beta_hat(df)

In [ ]:
# Print least squares estimate values
print(f"Least squares estimates:\nc0: {beta_hat[0]:.4f}\ns: {beta_hat[1]:.4f}")

## Bayesian Linear Regression

Instead of treating the intercept and slope terms as constants and obtaining their point estimates, it is possible instead to calibrate the parameters in a Bayesian framework.  If a certain prior distribution is assumed for the parameters $(\beta,\sigma^2)$ and the measurement error model is Gaussian, then the posterior distribution of $\beta$ is a bivariate t-distribution.  In particular, if the prior distribution on the parameters is
$$
p(\beta,\sigma^2)\propto\frac{1}{\sigma^2}
$$
and the measurement model is
$$
Y|\beta,\sigma^2\sim N(X\beta,\sigma^2I_n),
$$
then the marginal posterior distribution of $\beta$ is
$$
p(\beta|Y)=\frac{\Gamma((\nu+p)/2)}{\Gamma(\nu/2)\nu^{p/2}\pi^{p/2}|\Sigma|^{1/2}}\left(1+\frac{1}{\nu}(\beta-\hat{\beta})'\Sigma^{-1}(\beta-\hat{\beta})\right)^{-n/2},
$$
where $\nu=n-2$ is the degrees of freedom and the posterior mean and shape matrix are
\begin{align}
\hat{\beta}&=(X'X)^{-1}X'Y \\
\Sigma&=s^2(X'X)^{-1}.
\end{align}
Here,
$$
s^2=\frac{1}{\nu}\|Y-\hat{Y}\|^2
$$
is the sample variance and $\hat{Y}=X\hat{\beta}$ are the predicted shock wave velocities.
The marginal posterior distribution of $\sigma^2$ is an inverse gamma distribution,
$$
\sigma^2|Y \sim \textrm{InvGamma}\left(\frac{\nu}{2},\frac{\nu s^2}{2}\right).
$$
Hence, to obtain the marginal posterior distributions of $\beta$ and $\sigma^2$, we need to compute $\hat{\beta}$, $\Sigma$, and $s^2$.

The expressions for the posterior distribution are derived in Banerjee (2008), for example.

In [ ]:
# Compute posterior distribution
beta_hat, Sigma, s_sq, nu = calibration.compute_posterior_parameters(df)

The posterior covariance matrix of $\beta$ is $\frac{\nu}{(\nu-2)}\Sigma$, not $\Sigma$.

In [ ]:
# Compute posterior covariance matrix
posterior_covariance_matrix = nu / (nu - 2) * Sigma

In [ ]:
# Print posterior mean
print("Posterior Mean:", beta_hat)

In [ ]:
# Compute posterior standard deviations and covariance
posterior_sd_C0 = np.sqrt(posterior_covariance_matrix[0, 0])
posterior_sd_S = np.sqrt(posterior_covariance_matrix[1, 1])
posterior_covariance_C0_S = posterior_covariance_matrix[0, 1]

In [ ]:
# Print posterior standard deviations and covariance
print(f"Posterior standard deviation of C0: {posterior_sd_C0:.5f}")
print(f"Posterior standard deviation of S: {posterior_sd_S:.5f}")
print(f"Posterior covariance of C0 and S: {posterior_covariance_C0_S:.4f}")

The marginal posterior distribution of $\beta$ is centered at $\hat{\beta}$ and the contours are ellipses.

In [ ]:
# Create a contour plot of the posterior density of beta
calibration.plot_joint_posterior(beta_hat, Sigma, nu)

## Markov Chain Monte Carlo (MCMC)

MCMC is a technique that can be used for drawing samples from the posterior distribution of $\beta$ and $\sigma^2$.  There are many different MCMC samplers, including the Metropolis algorithm, Metropolis-Hasting algorithm, and the No U-Turn Sampler (NUTS).  There are additionally many Python packages that implement these samplers, including [Stan](https://mc-stan.org/) and [PyMC](https://www.pymc.io/projects/docs/en/stable/learn.html).  This notebook uses a popular package called [emcee](https://emcee.readthedocs.io/en/stable/) that implements an affine invariant ensemble sampling technique.

In [ ]:
# Run MCMC and get the samples and acceptance rates
samples, acceptance_rates = mcmc.get_samples(
    df=df,
    nsteps=50_000,
    nwalkers=6,
    burn_in=10_000,
    thin=5,
)

In [ ]:
# Print dimensions of MCMC samples array
print(f"MCMC samples array has shape {samples.shape}")

In [ ]:
# Create a trace plot of the samples for the first walker
mcmc.trace_plot(samples[:, 0, :])

In [ ]:
# Plot histograms of the MCMC samples of the regression coefficients w/ actual density overlaid
stacked_samples = samples.reshape(-1, samples.shape[-1])
mcmc.plot_samples_with_marginals(stacked_samples, beta_hat, Sigma, nu)

In [ ]:
# Plot MCMC samples of the variance parameter with the actual posterior density overlaid
mcmc.plot_marginal_density_sigma_sq(df, s_sq, stacked_samples)

In [ ]:
# Pairs plot and marginals of MCMC samples
mcmc.posterior_pairs_plot(stacked_samples, beta_hat)

In [ ]:
# Compute estimate of posterior mean of C0, S, sigma^2
stacked_samples.mean(axis=0)

In [ ]:
# Compute estimate of posterior covariance variance from chains, assumes independence
np.cov(stacked_samples.T)

After obtaining MCMC samples, it is important to assess diagnostics such as the acceptance rate and autocorrelation of the samples. Accounting for autocorrelation is important when computing a confidence interval for a Monte Carlo estimate obtained with the MCMC samples. See sec. 3 of Flegal et al. (2008) for related discussion.

In [ ]:
# Plot autocorrelation function (ACF) of MCMC chains for first walker
mcmc.plot_acf_of_chains(samples, nlags=40)

In [ ]:
# Print acceptance rate of sampler
print(f"Acceptance rates: {acceptance_rates}")

## Bootstrapping

Bootstrapping consists of repeating the following steps:

1. Sample the data with replacement, so duplicate $(U_p,U_s)$ pairs can possibly be in this new data set
2. Fit the linear model to this new data set

The set of fitted regression coefficients is called a bootstrap distribution, and approximates the distribution of regression coefficients over different possible data sets.  This is in contrast to the posterior distribution obtained in a Bayesian framework, which represents the distribution of regression coefficients for the observed data.

In [ ]:
# Sample bootstrap distribution of (c0, s)
bootstrap_samples = bootstrap.draw_samples(df, n=20_000)

In [ ]:
# Plot bootstrap distributions of c0 and s
bootstrap.plot_marginal_bootstrap_distributions(bootstrap_samples)

In [ ]:
# Compute bootstrap mean
bootstrap_mean = bootstrap_samples.mean(axis=0)

In [ ]:
# Print bootstrap mean
print(f"Bootstrap mean for C0: {bootstrap_mean[0]:.4f}")
print(f"Bootstrap mean for S: {bootstrap_mean[1]:.4f}")

In [ ]:
# Compute bootstrap confidence intervals for C_0 and S
ci_lower, ci_upper = np.percentile(bootstrap_samples, [2.5, 97.5], axis=0)

In [ ]:
# Print bootstrap confidence intervals
print(f"Bootstrap CI for C0: ({ci_lower[0]:.4f}, {ci_upper[0]:.4f})")
print(f"Bootstrap CI for S: ({ci_lower[1]:.4f}, {ci_upper[1]:.4f})")

## Sampling Hugoniot in the Pressure-Volume Plane

The Rankine-Hugoniot equations are a set of three conservation equations that relate the particle and shock wave velocities to the pressure, volume, and internal energy of the material.  The conservation of mass equation is
$$
\frac{V}{V_0}=\frac{U_s-U_p}{U_s},
$$
where $V$ and $V_0$ are the volume behind the shock front and the initial volume, respectively. The conservation of momentum equation is
$$
P-P_0=\rho_0 U_s U_p,
$$
where $P_0$ and $\rho_0$ are the initial pressure and density of the material and $P$ is the pressure of the material behind the shock front.

Hugoniot curves can therefore be sampled in the pressure-volume plane with the following steps:

- Sample $\beta$ from the posterior bivariate t-distribution given above
- Compute the corresponding $U_s-U_p$ curve as $U_s=C_0 + S U_p$ over a grid of $U_p$ values
- Plug these values of $U_s$ and $U_p$ into the first two Rankine-Hugoniot equations to get the Hugoniot curve in the pressure-volume plane

In [ ]:
# Create a plot of Hugoniot curves in pressure-volume space
hugoniot.plot_Hugoniot_samples(df, beta_hat, Sigma, nu, n_sample=100)

## Discussion Questions

These questions require reflecting on the material presented in this notebook:

1. Bootstrapping and Bayesian calibration both provide distributions over $\beta$, but these distributions have different interpretations since the bootstrap distribution characterizes the uncertainty in the least squares model parameter estimates over different datasets, whereas the posterior distribution quantifies uncertainty in the model parameters given the observed data.  Given these two interpretations, can you think of examples where one distribution would be preferred over the other?

2. How could the Bayesian model be made more general?  Consider the covariance matrix of the measurement errors, for example.

3. For Bayesian linear regression as discussed here, MCMC is not necessary since the posterior distribution is analytically available.  Can you think of any reasons why one might still want to use MCMC in this case?

4. Your colleague has expertise in grid-based numerical methods and wants to perform a Bayesian analysis of linear shock compression data.  Can you suggest a grid-based method that could be used to obtain the posterior distribution of $(\beta,\sigma^2)$?

## Extension Questions

These questions ask you to modify the notebook or supporting code:

1. Modify the `eda.plot_raw_data` function to indicate the experimental type of each point.  Are any trends apparent?

2. Rerun the notebook without removing the point with the largest particle velocity.  How do the means and variances of the posterior and bootstrap distributions of $C_0$ and $S$ change?

3. Is the posterior correlation between $C_0$ and $S$ positive or negative?  Can you give an intuitive explanation why this is the case?

4. In the MCMC simulation, experiment with varying the thin parameter and observe how the autocorrelation function changes as a function of this parameter.  Also set the burn-in parameter to 0 and modify the code in `mcmc.get_samples` so that the initial values for the chains are more dispersed.  How does this appear to affect the convergence of the chains?

5. Read about the normal-inverse-gamma distribution on [this Wikipedia page](https://en.wikipedia.org/wiki/Normal-inverse-gamma_distribution).  Modify the `mcmc.log_prior` function to implement this distribution, which requires adding the parameters $\mu,V,\alpha$, and $\beta$.  Run the MCMC analysis with $\mu=(3.8,1.6)'$, $V=I_2$, $\alpha=0.5$, and $\beta=0.2$, or with other values you choose.  Do the MCMC samples from the posterior distribution of $\beta$ appear normal?  Explain why or why not.

## References

- Banerjee, S. (2008). Bayesian linear model: Gory details. [PDF Link](https://ams206-winter18-01.courses.soe.ucsc.edu/system/files/attachments/banerjee-Bayesian-Linear-Model-Gory-Details.pdf).
- Flegal, J. M., Haran, M., & Jones, G. L. (2008). Markov chain Monte Carlo: Can we trust the third significant figure?. Statistical Science, 250-260.
- Foreman-Mackey, D., Hogg, D. W., Lang, D., & Goodman, J. (2013). emcee: the MCMC hammer. Publications of the Astronomical Society of the Pacific, 125(925), 306.
- Marsh, S. P. (1980). LASL shock Hugoniot data (Vol. 5). Univ of California Press.
    - Copper data given on pages 57-60.
- Gelman, A., Carlin, J. B., Stern, H. S., & Rubin, D. B. (1995). Bayesian data analysis. Chapman and Hall/CRC.
    - See chapter 11 on Markov chain simulation and chapter 14 on regression models.
- Rencher, A. C., & Schaalje, G. B. (2008). Linear models in statistics. John Wiley & Sons.
    - See chapter 11 on Bayesian regression.